In [ ]:
import cdsapi
import os

folder_out = r'/home/madse/Dokumente/cds_Data'


lat = 47.99305
lon = 7.84068
point = str(lat) + '/' + str(lon) + '/' + str(lat) + '/' + str(lon)

# create a client object
c = cdsapi.Client()

# define the request parameters
year = ['2023']
months = [str(i).zfill(2) for i in range(1, 4)] #max 6 months
days = [str(i).zfill(2) for i in range(1, 32)]
variables_list = ['2m_temperature','10m_u_component_of_wind', '10m_v_component_of_wind']

# define the output file name by year day and month 
downloaded_file = 'ERA5-Land_' + year[0] + months[0] + days[0] +'_till_' + year[0] + months[-1] + days[-1] + '_wind.nc'
downloaded_file = os.path.join(folder_out, downloaded_file)

# send the request to the CDS server
c.retrieve(
    'reanalysis-era5-land',
    {
        'year': year,
        'month': months,
        'day': days,
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',],
        'variable': variables_list,
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'area': point, # extract data for a single point
    },
    downloaded_file
)


In [ ]:
import plotly.graph_objects as go
import xarray as xr
import netCDF4 as nc
import numpy as np
import datetime
import pandas as pd

# read the downloaded file
data = nc.Dataset(downloaded_file, 'r')

# extract the u and v wind components
time = data.variables['time'][:]
t2m = data.variables['t2m'][:]
u10 = data.variables['u10'][:] 
v10 = data.variables['v10'][:] 

# close the file
data.close()

# convert time to list of datetime objects
dates = []
for t in time:
    date = datetime.datetime(1900, 1, 1) + datetime.timedelta(hours=int(t))
    dates.append(date)

# get absolute wind speed
wind_speed_10m = (u10**2 + v10**2)**0.5

# Calculate wind speed at 100m using power law
height = 100.0
h = height - 10.0
alpha = 0.143  # typical value for neutral conditions
wind_speed_100m = wind_speed_10m * (h / 10.0) ** alpha

# # Calculate wind speed at 100m using logarithmic law
# z0 = 0.1
# kappa = 0.4
# ustar = wind_speed_10m / np.log(10.0 / z0)
# theta0 = np.arctan(0.5 * t2m / ustar)
# zeta = (np.log(h/z0) - psi_m(h/z0)) / kappa
# wind_speed_100m_log = ustar / kappa * np.log(height / z0) / np.cos(theta0) * (1 - zeta)


# create a dictionary with the variables
data_dict = {'dates': dates, 't2m': t2m.flatten(), 'wind_speed_10m': wind_speed_10m.flatten(), 'wind_speed_100m': wind_speed_100m.flatten(),'wind_speed_100m_log': wind_speed_100m_log.flatten()}
# create a pandas DataFrame with the dictionary
df = pd.DataFrame(data_dict)




In [ ]:
import numpy as np

# Define constants and parameters
air_density = 1.225  # kg/m^3
R = 287.058         # J/kg-K
T = 288.15          # K (15 degrees Celsius)
radius = 45         # m (half of rotor diameter)
swept_area = np.pi * radius**2  # m^2
cp = [0.00, 0.40, 0.45, 0.45, 0.46, 0.48, 0.49, 0.50, 0.50, 0.50, 0.50]  # obtained from power curve
cut_in_speed = 3     # m/s
rated_speed = 16     # m/s
cut_out_speed = 25   # m/s
rated_power = 1000   # kW

# Define function to calculate power output
def power_output(wind_speed):
    if wind_speed < cut_in_speed or wind_speed >= cut_out_speed:
        return 0
    elif wind_speed < rated_speed:
        i = int(np.floor((wind_speed - cut_in_speed) / (rated_speed - cut_in_speed) * 10))
        cp_i = cp[i]
        p = 0.5 * air_density * swept_area * cp_i * wind_speed**3 / 10000  # kW
        return p
    else:
        p = rated_power
        return p

# Example usage
wind_speed = 12.5  # m/s
power = power_output(wind_speed)

#add power to df as new column
df['power'] = np.zeros(len(df['wind_speed_100m']))

for i in range(len(df['wind_speed_100m'])):
    power = power_output(df['wind_speed_100m'][i])
    df.loc[i, 'power'] = power



In [ ]:
# Create a scatter plot of wind speed at 10m and 100m and power output in a second y-axis
import plotly.graph_objects as go
from plotly import tools
#define figure with second axis

# use plotly.tools.make_subplots to create the figure with a subplot grid
fig = tools.make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.001,specs=[[{"secondary_y": True}]])

#fig = go.Figure()
# add first trace to first axis
fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_speed_10m'], name='10m'))
# add second trace to first axis
fig.add_trace(go.Scatter(x=df['dates'], y=df['wind_speed_100m'], name='100m'))
# add third trace to second axis
fig.add_trace(go.Scatter(x=df['dates'], y=df['power'], name='power', yaxis='y2'))
# set first axis title
fig.update_yaxes(title_text='Wind Speed (m/s)', secondary_y=False)
# set second axis title
fig.update_yaxes(title_text='Power (kW)', secondary_y=True)

# Set the plot title and axes labels
fig.update_layout(title='Wind Speed at 10m and 100m', xaxis_title='Time', yaxis_title='Wind Speed (m/s)')

# Display the plot
fig.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define constants and parameters
air_density = 1.225  # kg/m^3
R = 287.058         # J/kg-K
T = 288.15          # K (15 degrees Celsius)
radius = 45         # m (half of rotor diameter)
swept_area = np.pi * radius**2  # m^2
cp = [0.00, 0.40, 0.45, 0.45, 0.46, 0.48, 0.49, 0.50, 0.50, 0.50, 0.50]  # obtained from power curve
cut_in_speed = 3     # m/s
rated_speed = 16     # m/s
cut_out_speed = 25   # m/s
rated_power = 1000   # kW

# Define function to calculate power output
def power_output(wind_speed):
    if wind_speed < cut_in_speed or wind_speed >= cut_out_speed:
        return 0
    elif wind_speed < rated_speed:
        i = int(np.floor((wind_speed - cut_in_speed) / (rated_speed - cut_in_speed) * 10))
        cp_i = cp[i]
        p = 0.5 * air_density * swept_area * cp_i * wind_speed**3 / 10000  # kW
        return p
    else:
        p = rated_power
        return p

# Create arrays of wind speeds and power output
wind_speeds = np.linspace(0, 30, 100)
powers = np.zeros_like(wind_speeds)

for i, wind_speed in enumerate(wind_speeds):
    powers[i] = power_output(wind_speed)

# Normalize wind speeds and powers
normalized_wind_speeds = wind_speeds / cut_out_speed
normalized_powers = powers / rated_power

# Plot power curve
fig, ax = plt.subplots()
ax.plot(normalized_wind_speeds, normalized_powers)
ax.set_xlabel('Normalized Wind Speed')
ax.set_ylabel('Normalized Power')
ax.set_title('Power Curve')
plt.show()
